In [206]:
import re
from urllib.parse import quote_plus
from nbformat import read
from nbconvert import MarkdownExporter
from nbconvert.writers import FilesWriter
from nbconvert.nbconvertapp import NbConvertApp
from nbconvert.preprocessors import Preprocessor

In [126]:
notebook = './1-MonteCarlo-Resource-Assessments-part1.ipynb'

In [710]:
with open(notebook, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
try:
    output = quote_plus(
        nb.metadata.date + '-' + nb.metadata.title, safe='/'
    )
except:
    output = ''

In [711]:
class Remove_H1(Preprocessor):
    """A Pelican specific preprocessor to remove some of the cells of a notebook"""

#     # I could also read the cells from nb.metadata.pelican if someone wrote a JS extension,
#     # but for now I'll stay with configurable value.
#     start = Integer(0,  help="first cell of notebook to be converted").tag(config=True)
#     end   = Integer(-1, help="last cell of notebook to be converted").tag(config=True)

    def preprocess(self, nb, resources):

        for cell in nb.cells:
            if cell.cell_type == 'markdown':
#                 s = re.search("^\# .+\n\n", cell.source)
                s = re.sub("^\# .+\n\n", '', cell.source, re.MULTILINE)
#                 cell.source = s
        
#                 pattern = r"\\\[(?!\\\[).*\\\]"
# #                 pattern = r"^(?!\\\]).*\\\]"
#                 pattern = r"(\\\[)((?:(?!\1)[\s\S])*)(\\\])"
# #                 s = re.findall(pattern, s, re.MULTILINE | re.DOTALL)
                pre = """<div class="equation">\n\t<div>"""
                post = """</div>\n<div class="equation_dots"></div>\n"""
                post += "<div></div>\n"
                post += "</div>\n"
                s = re.sub(r'\\\\\[', pre + r'\\[', s)
                s = re.sub(r'\\\\\]', r'\\]' + post, s)
                cell.source = s
        return nb, resources

In [712]:
md_exporter = MarkdownExporter(
    template_name="blog-markdown",
    preprocessors=[Remove_H1]
)
# md_exporter.preprocessors = [Remove_H1]

In [713]:
md_writer = FilesWriter(
    build_directory = "/Users/dom/Downloads/nbconvert_output",
)


In [714]:
app = NbConvertApp(
    output_base = output
)
app.exporter = md_exporter
app.writer = md_writer

In [715]:
app.convert_single_notebook(notebook)

[NbConvertApp] Converting notebook ./1-MonteCarlo-Resource-Assessments-part1.ipynb to 


In [709]:
md_exporter.preprocessors

[__main__.Remove_H1]

In [427]:
prog = re.compile('.', re.DOTALL | re.MULTILINE)